<a href="https://colab.research.google.com/github/mashuai191/pantograph/blob/master/Pantograph_Demo_segmentation%2Blocalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# how to run web app in google colab
    #refer to : https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b


In [0]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
#!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
#!pip install torch torchvision
!pip install numpy
!pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'


Looking in links: https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-gy47gwe4
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-gy47gwe4
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=275297 sha256=832f2aff2423c9645cad9457eb336507a4400792d0f0099a9389e8dbd7ed8e30
  Stored in directory: /tmp/pip-ephem-wheel-cache-lij94tbf/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


## install Detectron2

In [0]:

!rm -rf detectron2_repo
!git clone https://github.com/mashuai191/detectron2 detectron2_repo

!pip install -e detectron2_repo

%tensorflow_version 1.x


Cloning into 'detectron2_repo'...
remote: Enumerating objects: 3563, done.
remote: Total 3563 (delta 0), reused 0 (delta 0), pack-reused 3563
Receiving objects: 100% (3563/3563), 2.14 MiB | 1.91 MiB/s, done.
Resolving deltas: 100% (2458/2458), done.
Obtaining file:///content/detectron2_repo
  Found existing installation: detectron2 0.1.1
    Can't uninstall 'detectron2'. No files were found to uninstall.
  Running setup.py develop for detectron2
TensorFlow 1.x selected.


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

## mount gdrive for dataset and weight file

In [0]:

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## install yolo v3

In [0]:
!rm pantograph -rf
!git clone 'https://github.com/mashuai191/pantograph.git'

Cloning into 'pantograph'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 92 (delta 29), reused 62 (delta 11), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [0]:
!cp '/content/gdrive/My Drive/App/machine_learning/cv/shanghaidianqi_line5/yolo3/qingsong/yolo.h5' /content/pantograph/localization_yolo3/model_data

In [0]:
%tensorflow_version 1.x

import sys
sys.path.append('/content/pantograph/localization_yolo3')

from yolo import YOLO

yolo = YOLO()

Using TensorFlow backend.














/content/pantograph/localization_yolo3/model_data/yolo.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
!pip list | grep tensorflow

mesh-tensorflow          0.1.12          
tensorflow               1.15.2          
tensorflow-addons        0.8.3           
tensorflow-datasets      2.1.0           
tensorflow-estimator     1.15.1          
tensorflow-gan           2.0.0           
tensorflow-gcs-config    2.1.8           
tensorflow-hub           0.7.0           
tensorflow-metadata      0.21.1          
tensorflow-privacy       0.2.2           
tensorflow-probability   0.7.0           


In [0]:
!pip install flask-ngrok


In [0]:
import atexit
import json
import os
import platform
import shutil
import subprocess
import tempfile
import time
import zipfile
from pathlib import Path
from threading import Timer

import requests


def _get_command():
    system = platform.system()
    if system == "Darwin":
        command = "ngrok"
    elif system == "Windows":
        command = "ngrok.exe"
    elif system == "Linux":
        command = "ngrok"
    else:
        raise Exception("{system} is not supported".format(system=system))
    return command


def _run_ngrok(port):
    command = _get_command()
    ngrok_path = str(Path(tempfile.gettempdir(), "ngrok"))
    _download_ngrok(ngrok_path)
    executable = str(Path(ngrok_path, command))
    os.chmod(executable, 0o777)
    ngrok = subprocess.Popen([executable, 'http', str(port)])
    atexit.register(ngrok.terminate)
    localhost_url = "http://localhost:4040/api/tunnels"  # Url with tunnel details
    time.sleep(1)
    tunnel_url = requests.get(localhost_url).text  # Get the tunnel information
    j = json.loads(tunnel_url)

    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
    tunnel_url = tunnel_url.replace("https", "http")
    return tunnel_url


def _download_ngrok(ngrok_path):
    if Path(ngrok_path).exists():
        return
    system = platform.system()
    if system == "Darwin":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-darwin-amd64.zip"
    elif system == "Windows":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-windows-amd64.zip"
    elif system == "Linux":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"
    else:
        raise Exception(f"{system} is not supported")
    download_path = _download_file(url)
    with zipfile.ZipFile(download_path, "r") as zip_ref:
        zip_ref.extractall(ngrok_path)


def _download_file(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    download_path = str(Path(tempfile.gettempdir(), local_filename))
    with open(download_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)
    return download_path


def start_ngrok(port):
    ngrok_address = _run_ngrok(port)
    print(f" * Running on {ngrok_address}")
    print(f" * Traffic stats available on http://127.0.0.1:4040")


def run_with_ngrok(app):
    """
    The provided Flask app will be securely exposed to the public internet via ngrok when run,
    and the its ngrok address will be printed to stdout
    :param app: a Flask application object
    :return: None
    """
    old_run = app.run

    def new_run(*args, **kwargs):
        port = kwargs.get('port', 5000)
        thread = Timer(1, start_ngrok, args=(port,))
        thread.setDaemon(True)
        thread.start()
        old_run(*args, **kwargs)
    app.run = new_run

In [0]:
#from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"
  
#app.run()

In [0]:
import cv2
import os

DIR_BASE = '/content/gdrive/My Drive/App/machine_learning/cv/shanghaidianqi_line5'
DIR_DATASET = '20181117_03_08_1750_G1380.MP4'
DIR_DATASET2 = 'night.mp4'



In [0]:
%cd /content/

/content


In [0]:
!rm -rf flask-video-streaming
!git clone https://github.com/mashuai191/flask-video-streaming.git

Cloning into 'flask-video-streaming'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 99 (delta 6), reused 4 (delta 1), pack-reused 79
Unpacking objects: 100% (99/99), done.


In [0]:
%cd flask-video-streaming

/content/flask-video-streaming


In [0]:
!ls


1.jpg	base_camera.py	  LICENSE		   requirements.txt
2.jpg	camera_opencv.py  README.md		   templates
3.jpg	camera_pi.py	  requirements-opencv.txt
app.py	camera.py	  requirements-pi.txt


In [0]:
import os
import cv2
from base_camera import BaseCamera

class segCamera(BaseCamera):
    video_source = 0

    def __init__(self):
        if os.environ.get('OPENCV_CAMERA_SOURCE'):
            segCamera.set_video_source(int(os.environ['OPENCV_CAMERA_SOURCE']))
        super(segCamera, self).__init__()

    @staticmethod
    def set_video_source(source):
        segCamera.video_source = source

    @staticmethod
    def frames():
        #camera = cv2.VideoCapture(Camera.video_source)
        camera = cv2.VideoCapture(os.path.join(DIR_BASE, DIR_DATASET))
        if not camera.isOpened():
            raise RuntimeError('Could not start camera.')

        #import torch
        import multiprocessing
        print("11111", multiprocessing.get_start_method())
        #torch.multiprocessing.set_sharing_strategy('file_system')
        
        MetadataCatalog.get("balloon_" + 'test').set(thing_classes=["balloon"])
        cfg = get_cfg()
        # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
        # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
        #cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        #cfg.OUTPUT_DIR = "pre-trained models/model_final_R50_x3_ratio012.pth"
        cfg.MODEL.WEIGHTS = os.path.join(DIR_BASE, "trained_models/model_final_R50_x3_ratio012.pth")
        cfg.DATASETS.TEST = ("balloon_test", )
        cfg.DATASETS.TRAIN = ("balloon_test", )
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

        predictor = DefaultPredictor(cfg)

        print("22222", torch.multiprocessing.get_start_method())


        while True:
            # read current frame
            _, img = camera.read()
            cropped_image = img[200:625, 200:1150]
            print ("seg:", img.shape, cropped_image.shape)
            

            # let's predict

            #outputs = predictor(cropped_image)

            #v = Visualizer(cropped_image[:, :, ::-1], metadata=MetadataCatalog.get("balloon_" + 'test'), scale=1.2)
            #v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            #img_tmp = v.get_image()[:, :, ::-1]
            #cv2_imshow(v.get_image()[:, :, ::-1])


            #imS = cv2.resize(img_tmp, (350, 250))
            imS = cv2.resize(img, (100, 50))
            #print (imS.shape)

            # encode as a jpeg image and return it
            yield cv2.imencode('.jpg', imS)[1].tobytes()


In [0]:
import os
import cv2
import numpy as np


from PIL import Image, ImageFont, ImageDraw

from base_camera import BaseCamera


class locCamera(BaseCamera):
    video_source = 0

    def __init__(self):
        if os.environ.get('OPENCV_CAMERA_SOURCE'):
            locCamera.set_video_source(int(os.environ['OPENCV_CAMERA_SOURCE']))
        super(locCamera, self).__init__()

    @staticmethod
    def set_video_source(source):
        locCamera.video_source = source

    @staticmethod
    def frames():
        #camera = cv2.VideoCapture(Camera.video_source)
        camera = cv2.VideoCapture(os.path.join(DIR_BASE, DIR_DATASET2))
        if not camera.isOpened():
            raise RuntimeError('Could not start camera.')

        while True:
            # read current frame
            _, img = camera.read()

            # = img[200:625, 200:1150]
            
            # let's predict
            image = Image.fromarray(img)
            image = yolo.detect_image(image)
            result = np.asarray(image)

            #v = Visualizer(cropped_image[:, :, ::-1], metadata=MetadataCatalog.get("balloon_" + 'test'), scale=1.2)
            #v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            #img_tmp = v.get_image()[:, :, ::-1]
            #cv2_imshow(v.get_image()[:, :, ::-1])


            imS = cv2.resize(result, (350, 250))
            #imS = cv2.resize(img, (200, 100))
            print ("loc:", img.shape, imS.shape)

            # encode as a jpeg image and return it
            yield cv2.imencode('.jpg', imS)[1].tobytes()


In [0]:
#!/usr/bin/env python
from importlib import import_module
import os
from flask import Flask, render_template, Response
import multiprocessing as mp
mp.set_start_method('spawn',force=True)
    


# import camera driver
if os.environ.get('CAMERA'):
    Camera = import_module('camera_' + os.environ['CAMERA']).Camera
else:
    from camera import Camera

# Raspberry Pi camera module (requires picamera package)
# from camera_pi import Camera
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run


@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')


def gen(camera):
    """Video streaming generator function."""
    while True:
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    print ('video_feed, pid {}'.format(os.getpid()))
    import torch
    print("reg process", torch.multiprocessing.get_start_method())
    #torch.multiprocessing.set_start_method('forkserver', force=True)

    return Response(gen(segCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed1')
def video_feed1():
    """Video streaming route. Put this in the src attribute of an img tag."""
    print ('video_feed1, pid {}'.format(os.getpid()))
    
    #torch.multiprocessing.set_start_method('spawn', force=True)
    return Response(gen(Camera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

#if __name__ == '__main__':
#    app.run(host='0.0.0.0', threaded=True)
#from flask.flask_script import Manager

#manager = Manager(app)
    

if __name__ == "__main__":
#    manager.run(host='0.0.0.0', threaded=True)
    print ('main, pid {}'.format(os.getpid()))

    #import torch
    #torch.multiprocessing.set_start_method('forkserver', force=True)
    #mp.set_start_method('spawn',force=True)
    print (mp.get_start_method())
    #mp.set_start_method('spawn')
    #mp = torch.multiprocessing.get_context('spawn')
    #torch.multiprocessing.set_start_method('spawn')
    #torch.multiprocessing.set_start_method('forkserver', force=True)
    app.run(host='0.0.0.0', threaded=False, processes=2)


main, pid 1382
spawn
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


 * Running on http://37115fa7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Apr/2020 14:40:50] "GET / HTTP/1.1" 200 -


video_feed, pid 1741
reg process spawn
Camera init.
pid  1741 tid  140156196087680
Starting camera thread.
video_feed1, pid 1744
Camera init.
pid  1744 tid  140156196087680
Starting camera thread.
11111 spawn


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/content/flask-video-streaming/base_camera.py", line 97, in _thread
    for frame in frames_iterator:
  File "<ipython-input-16-95b269c1163b>", line 42, in frames
    predictor = DefaultPredictor(cfg)
  File "/content/detectron2_repo/detectron2/engine/defaults.py", line 163, in __init__
    self.model = build_model(self.cfg)
  File "/content/detectron2_repo/detectron2/modeling/meta_arch/build.py", line 19, in build_model
    return META_ARCH_REGISTRY.get(meta_arch)(cfg)
  File "/content/detectron2_repo/detectron2/modeling/meta_arch/rcnn.py", line 41, in __init__
    pixel_mean = torch.Tensor(cfg.MODEL.PIXEL_MEAN).to(self.device).view(num_channels, 1, 1)
  File "/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py", l

video_feed, pid 1771
reg process spawn
Camera init.
pid  1771 tid  140156196087680
Starting camera thread.
11111 spawn


127.0.0.1 - - [06/Apr/2020 14:40:54] "GET /video_feed1 HTTP/1.1" 200 -
Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/content/flask-video-streaming/base_camera.py", line 97, in _thread
    for frame in frames_iterator:
  File "<ipython-input-16-95b269c1163b>", line 42, in frames
    predictor = DefaultPredictor(cfg)
  File "/content/detectron2_repo/detectron2/engine/defaults.py", line 163, in __init__
    self.model = build_model(self.cfg)
  File "/content/detectron2_repo/detectron2/modeling/meta_arch/build.py", line 19, in build_model
    return META_ARCH_REGISTRY.get(meta_arch)(cfg)
  File "/content/detectron2_repo/detectron2/modeling/meta_arch/rcnn.py", line 41, in __init__
    pixel_mean = torch.Tensor(cfg.MODEL.PIXEL_MEAN).to(self.device).view(num_channels, 1, 1)
  

In [0]:
!nvidia-smi

Mon Apr  6 14:40:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    31W / 250W |   1323MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# 一些发现

1. if in main: set_start_method('spawn', force=True), main和 fork的子进程里都是spawn了，但是

```
  File "/content/detectron2_repo/detectron2/modeling/meta_arch/rcnn.py", line 41, in __init__
    pixel_mean = torch.Tensor(cfg.MODEL.PIXEL_MEAN).to(self.device).view(num_channels, 1, 1)
  File "/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py", line 195, in _lazy_init
    "Cannot re-initialize CUDA in forked subprocess. " + msg)
RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

```



2. if in sub-process: torch.multiprocessing.set_start_method, 报错一样的



```
RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

```

3. 在main里面用 python 的 multiprocessing.set_start_method, 和2 报错一样

4. 在main 外面global的设置 

```
import multiprocessing as mp
mp.set_start_method('spawn',force=True)
```

报错


```
  File "<ipython-input-22-95b269c1163b>", line 42, in frames
    predictor = DefaultPredictor(cfg)
  File "/content/detectron2_repo/detectron2/engine/defaults.py", line 163, in __init__
    self.model = build_model(self.cfg)
  File "/content/detectron2_repo/detectron2/modeling/meta_arch/build.py", line 19, in build_model
    return META_ARCH_REGISTRY.get(meta_arch)(cfg)
  File "/content/detectron2_repo/detectron2/modeling/meta_arch/rcnn.py", line 41, in __init__
    pixel_mean = torch.Tensor(cfg.MODEL.PIXEL_MEAN).to(self.device).view(num_channels, 1, 1)
  File "/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py", line 197, in _lazy_init
    torch._C._cuda_init()
RuntimeError: cuda runtime error (3) : initialization error at /pytorch/aten/src/THC/THCGeneral.cpp:54
```





